# Problem statement

The dataset contains 36733 instances of 11 sensor measures aggregated over one hour (by means of average or sum) from a gas turbine. 
The Dataset includes gas turbine parameters (such as Turbine Inlet Temperature and Compressor Discharge pressure) in addition to the ambient variables.

We have to predict turbine energy yield (TEY) using ambient variables as features

In [5]:
#!pip install keras
#!pip install tensorflow

In [31]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV, KFold,train_test_split
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense

In [7]:
turbine = pd.read_csv("gas_turbines.csv")

In [8]:
turbine

,AT,AP,AH,AFDP,GTEP,TIT,TAT,TEY,CDP,CO,NOX
0,6.8594,1007.9,96.799,3.5000,19.663,1059.2,550.00,114.70,10.605,3.1547,82.722
1,6.7850,1008.4,97.118,3.4998,19.728,1059.3,550.00,114.72,10.598,3.2363,82.776
2,6.8977,1008.8,95.939,3.4824,19.779,1059.4,549.87,114.71,10.601,3.2012,82.468
3,7.0569,1009.2,95.249,3.4805,19.792,1059.6,549.99,114.72,10.606,3.1923,82.670
4,7.3978,1009.7,95.150,3.4976,19.765,1059.7,549.98,114.72,10.612,3.2484,82.311
...,...,...,...,...,...,...,...,...,...,...,...
15034,9.0301,1005.6,98.460,3.5421,19.164,1049.7,546.21,111.61,10.400,4.5186,79.559
15035,7.8879,1005.9,99.093,3.5059,19.414,1046.3,543.22,111.78,10.433,4.8470,79.917
15036,7.2647,1006.3,99.496,3.4770,19.530,1037.7,537.32,110.19,10.483,7.9632,90.912
15037,7.0060,1006.8,99.008,3.4486,19.377,1043.2,541.24,110.74,10.533,6.2494,93.227


In [9]:
turbine.shape

(15039, 11)

In [10]:
turbine.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15039 entries, 0 to 15038
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   AT      15039 non-null  float64
 1   AP      15039 non-null  float64
 2   AH      15039 non-null  float64
 3   AFDP    15039 non-null  float64
 4   GTEP    15039 non-null  float64
 5   TIT     15039 non-null  float64
 6   TAT     15039 non-null  float64
 7   TEY     15039 non-null  float64
 8   CDP     15039 non-null  float64
 9   CO      15039 non-null  float64
 10  NOX     15039 non-null  float64
dtypes: float64(11)
memory usage: 1.3 MB


<b>No null values are there in dataset</b>

In [11]:
turbine[turbine.duplicated()]

,AT,AP,AH,AFDP,GTEP,TIT,TAT,TEY,CDP,CO,NOX


<b>No duplicate rows are there in dataset</b>

# Train-test split

In [12]:
X = turbine.drop(["TEY"], axis = 1)
           
Y = turbine['TEY']

In [15]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = .3, random_state = 30)

In [16]:
X_train

,AT,AP,AH,AFDP,GTEP,TIT,TAT,CDP,CO,NOX
8657,10.3710,1013.2,79.248,5.3459,32.504,1100.0,529.88,13.927,0.53957,69.077
1247,8.8245,1025.3,84.123,3.9071,19.568,1047.9,543.61,10.564,5.55750,104.330
11304,26.1200,1013.8,64.292,3.7668,25.472,1091.6,549.91,12.181,0.72197,63.585
8857,5.7492,1016.4,84.679,4.2248,23.888,1087.2,550.00,12.013,0.33097,83.428
8812,5.7551,1012.5,92.997,5.3251,32.578,1100.0,528.18,14.113,0.79035,66.038
...,...,...,...,...,...,...,...,...,...,...
13452,23.1540,1014.0,71.004,4.4694,28.885,1100.4,543.40,12.992,1.14260,61.138
500,2.8258,1009.1,81.773,5.0467,29.647,1099.9,541.14,13.210,0.16464,79.371
12077,31.3320,1014.2,49.275,4.4837,29.665,1099.5,541.85,13.211,0.87442,58.806
4517,28.5630,1009.9,64.346,4.3587,26.438,1094.3,549.97,12.290,0.48279,62.532


In [17]:
Y_train

8657     160.70
1247     111.70
11304    133.80
8857     134.46
8812     164.57
          ...  
13452    146.73
500      156.47
12077    146.47
4517     134.75
5925     133.64
Name: TEY, Length: 10527, dtype: float64

In [18]:
X_test

,AT,AP,AH,AFDP,GTEP,TIT,TAT,CDP,CO,NOX
3390,29.2670,1012.4,43.795,4.2298,26.061,1093.2,550.06,12.206,1.64120,71.559
1194,11.5230,1024.4,88.089,4.3647,22.037,1069.2,548.48,11.221,2.42510,76.116
13890,15.2260,1019.0,84.659,3.8886,24.045,1087.1,549.91,11.938,1.70650,75.188
5570,27.3820,1016.1,61.555,4.4794,26.085,1093.7,550.04,12.337,1.38720,66.656
6190,19.1010,1008.6,83.062,4.3643,25.352,1089.7,550.16,12.030,2.47550,68.209
...,...,...,...,...,...,...,...,...,...,...
2257,18.9440,1014.1,89.131,5.1693,24.526,1085.5,550.14,11.883,4.45670,66.682
5628,19.5830,1015.7,96.398,4.9962,26.867,1098.8,549.54,12.571,0.90150,68.560
11140,23.4840,1013.7,83.599,3.7478,25.292,1090.6,550.39,12.123,0.36809,68.403
14565,7.0032,1024.3,83.088,3.7814,22.643,1078.7,550.09,11.591,2.13770,83.309


In [19]:
Y_test

3390     132.78
1194     122.78
13890    133.78
5570     133.76
6190     134.26
          ...  
2257     131.16
5628     141.54
11140    133.82
14565    130.25
8048     160.95
Name: TEY, Length: 4512, dtype: float64

In [25]:
norm = MinMaxScaler().fit(X_train)

In [26]:
X_train_norm = norm.transform(X_train)

In [27]:
X_test_norm = norm.transform(X_test)

In [28]:
Y_train = np.array(Y_train)

In [29]:
Y_test = np.array(Y_test)

# Using Neural network for prediction

In [20]:
def baseline_model():
    model = Sequential()
    model.add(Dense(13, input_dim=10, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal'))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [23]:
estimator = KerasRegressor(build_fn=baseline_model, epochs=500, batch_size=100, verbose=1)

In [32]:
estimator.fit(X_train_norm,Y_train)

Epoch 1/500
106/106 [==============================] - 1s 1ms/step - loss: 18175.1328
Epoch 2/500
106/106 [==============================] - 0s 1ms/step - loss: 17656.0000
Epoch 3/500
106/106 [==============================] - 0s 1ms/step - loss: 16553.6367
Epoch 4/500
106/106 [==============================] - 0s 1ms/step - loss: 14973.3877
Epoch 5/500
106/106 [==============================] - 0s 1ms/step - loss: 13056.3770
Epoch 6/500
106/106 [==============================] - 0s 1ms/step - loss: 10957.8711
Epoch 7/500
106/106 [==============================] - 0s 1ms/step - loss: 8830.6396
Epoch 8/500
106/106 [==============================] - 0s 2ms/step - loss: 6811.3232
Epoch 9/500
106/106 [==============================] - 0s 1ms/step - loss: 5008.7808
Epoch 10/500
106/106 [==============================] - 0s 1ms/step - loss: 3496.6687
Epoch 11/500
106/106 [==============================] - 0s 1ms/step - loss: 2309.5505
Epoch 12/500
106/106 [==============================] - 0

106/106 [==============================] - 0s 2ms/step - loss: 1.3315
Epoch 194/500
106/106 [==============================] - 0s 2ms/step - loss: 1.2825
Epoch 195/500
106/106 [==============================] - 0s 2ms/step - loss: 1.2338
Epoch 196/500
106/106 [==============================] - 0s 2ms/step - loss: 1.1956
Epoch 197/500
106/106 [==============================] - 0s 2ms/step - loss: 1.1512
Epoch 198/500
106/106 [==============================] - 0s 2ms/step - loss: 1.1175
Epoch 199/500
106/106 [==============================] - 0s 2ms/step - loss: 1.0814
Epoch 200/500
106/106 [==============================] - 0s 2ms/step - loss: 1.0471
Epoch 201/500
106/106 [==============================] - 0s 2ms/step - loss: 1.0195
Epoch 202/500
106/106 [==============================] - 0s 2ms/step - loss: 0.9871
Epoch 203/500
106/106 [==============================] - 0s 2ms/step - loss: 0.9628
Epoch 204/500
106/106 [==============================] - 0s 2ms/step - loss: 0.9425
Epoch 

106/106 [==============================] - 0s 1ms/step - loss: 0.6769
Epoch 387/500
106/106 [==============================] - 0s 1ms/step - loss: 0.6771
Epoch 388/500
106/106 [==============================] - 0s 957us/step - loss: 0.6784
Epoch 389/500
106/106 [==============================] - 0s 1ms/step - loss: 0.6760
Epoch 390/500
106/106 [==============================] - 0s 948us/step - loss: 0.6796
Epoch 391/500
106/106 [==============================] - 0s 1ms/step - loss: 0.6804
Epoch 392/500
106/106 [==============================] - 0s 948us/step - loss: 0.6799
Epoch 393/500
106/106 [==============================] - 0s 1ms/step - loss: 0.6770
Epoch 394/500
106/106 [==============================] - 0s 1ms/step - loss: 0.6797
Epoch 395/500
106/106 [==============================] - 0s 1ms/step - loss: 0.6755
Epoch 396/500
106/106 [==============================] - 0s 1ms/step - loss: 0.6748
Epoch 397/500
106/106 [==============================] - 0s 1ms/step - loss: 0.6726


### Predicting the test data

In [33]:
y_pred = estimator.predict(X_test_norm)

46/46 [==============================] - 0s 912us/step


In [34]:
y_pred.shape

(4512,)

### RMSE value for Test data

In [35]:
from sklearn.metrics import mean_squared_error
rms = mean_squared_error(Y_test, y_pred, squared=False)

In [36]:
rms

0.801902568310216

### RMSE value for Train data

In [37]:
y_pred_train = estimator.predict(X_train_norm)

106/106 [==============================] - 0s 1ms/step


In [38]:
rms_train = mean_squared_error(Y_train, y_pred_train, squared=False)
rms_train

0.8050133478493262

#### We could see there is not much difference in RMSE values for train and test data so we can say there is no overfitting problem in our model and we can finalize